### 训练集和测试集特征分离后   baseline : 0.713773

1、单独加入广告发布的不同用户数，0.715398，效果有较大提升

2、加入广告的出现次数，0.713612,效果下降

2、加入广告点击率和广告发布的不同用户数， 0.714031, 效果下降。。

2、加入campaign的点击率，0.715376，效果下降

3、加入用户所在LBS的历史点击率, 效果下降。。

4、加入用户的兴趣总数，0.714661，效果下降。。

5、加入用户的历史接收的广告次数与历史点击次数，以及历史点击率, 效果均下降

6、分析广告投放和地理位置的关系

7、加入每个地理位置的人数特征, 效果下降

8、考虑点击相同广告的用户的interest2分布,在点击的用户中，某个兴趣id出现次数是总人数的三分之一则保留  最终AUC为0.715613，没改变之前为0.715236

9、考虑增加字段，num_common_interest2，表示某个用户的兴趣组2中有多少与这条广告的所有兴趣id相符合 最终AUC为0.715452,之前为0.715326,
   继续增加字段, num_common_interest1, 最终AUC为0.715673，有提升,继续增加字段num_common_interest5，最终AUC为0.71563，有提升
   增加num_common_all字段，最终AUC为0.715714,有提升
   
10、考虑将数目变为比例值，结果参考9

11、增加topic1字段，0.716075，有提升, 加入关键字kw1,kw2下降,加入主题topic2也下降

12、尝试调低ratio到0.05，

In [1]:
import numpy as np
import pandas as pd
import os
from scipy import sparse
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
import xgboost as xgb
import lightgbm as lgb
import matplotlib.pyplot as plt

data = pd.read_csv('./datasets/train_data.csv')
target = pd.read_csv('./datasets/train_target.csv', names=['label'], header=None)

data.drop(data.columns[[0]], axis=1, inplace=True)  #删除某列函数
target = target.reset_index(drop=True)

one_hot_feature=['creativeSize', 'LBS','age','carrier','consumptionAbility', 'education','gender','house','os','marriageStatus','advertiserId','campaignId', 'creativeId',
       'adCategoryId', 'productId', 'productType']
vector_feature=['interest1','interest2','interest5','kw1','kw2','topic1','topic2']



for feature in one_hot_feature:
    try:
        data[feature] = LabelEncoder().fit_transform(data[feature].apply(int))
    except:
        data[feature] = LabelEncoder().fit_transform(data[feature])

In [109]:
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1, test_size=0.3, random_state=42)
y = np.array(target).squeeze()
for train_index, test_index in split.split(data,  y):
    X_train = data.iloc[train_index]
    X_test = data.iloc[test_index]
    y_train = y[train_index]
    y_test = y[test_index]

In [110]:
data_clicked = X_train[y_train == 1]

In [111]:
# 处理联网类型特征
ct_train = X_train['ct'].values
ct_train = [m.split(' ') for m in ct_train]
ct_trains = []
for i in ct_train:
    index = [0, 0, 0, 0, 0]
    for j in i:
        index[int(j)] = 1
    ct_trains.append(index)

ct_test = X_test['ct'].values
ct_test = [m.split(' ') for m in ct_test]
ct_tests = []
for i in ct_test:
    index = [0, 0, 0, 0, 0]
    for j in i:
        index[int(j)] = 1
    ct_tests.append(index)


增加每个广告点击率特征

In [283]:
num_ad = X_train['aid'].value_counts().sort_index()
num_ad_clicked = data_clicked['aid'].value_counts().sort_index()

ratio = num_ad_clicked / num_ad

ratio_clicked = pd.DataFrame({
    'aid': ratio.index,
    'ratio_clicked' : ratio.values
})


In [284]:
X_train = pd.merge(X_train, ratio_clicked, on=['aid'], how='left')
X_test = pd.merge(X_test, ratio_clicked, on=['aid'], how='left')

增加每个广告出现次数

In [285]:
num_ad_post = X_train['aid'].value_counts().sort_index()

num_ad_post = pd.DataFrame({
    'aid': num_ad_post.index,
    'num_ad_post' : num_ad_post.values
})

In [286]:
X_train = pd.merge(X_train, num_ad_post, on=['aid'], how='left')
X_test = pd.merge(X_test, num_ad_post, on=['aid'], how='left')

增加每个广告推送给不同的用户数

In [112]:
num_advertise_touser = X_train.groupby('aid').uid.nunique()
num_advertise_touser = pd.DataFrame({
    'aid': num_advertise_touser.index,
    'num_advertise_touser' : num_advertise_touser.values
})
X_train = pd.merge(X_train, num_advertise_touser, on=['aid'], how='left')
X_test = pd.merge(X_test, num_advertise_touser, on=['aid'], how='left')

加入推广计划转化率

In [55]:
num_campaign = X_train['campaignId'].value_counts().sort_index()
num_campaign_clicked = data_clicked['campaignId'].value_counts().sort_index()

ratio_num_campaign = num_campaign_clicked / num_campaign

ratio_num_campaign = pd.DataFrame({
    'campaignId': ratio_num_campaign.index,
    'ratio_num_campaign' : ratio_num_campaign.values
})
X_train = pd.merge(X_train, ratio_num_campaign, on=['campaignId'], how='left')
X_test = pd.merge(X_test, ratio_num_campaign, on=['campaignId'], how='left')

加入用户兴趣总数

In [56]:
train_all_interest = X_train[['interest1', 'interest2', 'interest5', 'kw1', 'kw2', 'topic1', 'topic2']]
test_all_interest = X_test[['interest1', 'interest2', 'interest5', 'kw1', 'kw2', 'topic1', 'topic2']]
num_all_interest_train = []
num_all_interest_test = []
aids = []
train_array = np.array(train_all_interest)
test_array = np.array(test_all_interest)

for i in range(train_array.shape[0]):
    num = 0
    inter = train_array[i]
    for j in inter:
        inter_lis = j.split(' ')
        if inter_lis[0] == '-1':
            continue
        num += len(inter_lis)
    num_all_interest_train.append(num)
    
    
for i in range(test_array.shape[0]):
    num = 0
    inter = test_array[i]
    for j in inter:
        inter_lis = j.split(' ')
        if inter_lis[0] == '-1':
            continue
        num += len(inter_lis)
    num_all_interest_test.append(num)

In [57]:
num_all_interest_train = pd.DataFrame(num_all_interest_train, columns=['num_all_interest'])
num_all_interest_test = pd.DataFrame(num_all_interest_test, columns=['num_all_interest'])

X_train = pd.concat([X_train, num_all_interest_train], axis=1)
X_test = pd.concat([X_test, num_all_interest_test], axis=1)

加入地理位置所对应的用户人数

In [58]:
num_lbs_user = X_train.groupby('LBS').uid.nunique()
num_lbs_user = pd.DataFrame({
    'LBS': num_lbs_user.index,
    'num_lbs_user' : num_lbs_user.values
})
X_train = pd.merge(X_train, num_lbs_user, on=['LBS'], how='left')
X_test = pd.merge(X_test, num_lbs_user, on=['LBS'], how='left')

加入用户所在LBS的历史点击率

In [21]:
num_lbs = X_train.groupby('LBS').uid.nunique()
num_lbs_clicked = data_clicked.groupby('LBS').uid.nunique()
ratio_num_lbs = num_lbs_clicked / num_lbs
ratio_num_lbs = ratio_num_lbs.fillna(0)
ratio_num_lbs = pd.DataFrame({
    'LBS': ratio_num_lbs.index,
    'ration_num_LBS' : ratio_num_lbs.values
})
X_train = pd.merge(X_train, ratio_num_lbs, on=['LBS'], how='left')
X_test = pd.merge(X_test, ratio_num_lbs, on=['LBS'], how='left')

加入用户的历史接收的广告次数与历史点击次数，以及历史点击率

In [15]:
num_user_getad = X_train.groupby('uid').aid.nunique().sort_index()
num_user_clicked = data_clicked.groupby('uid').aid.nunique().sort_index()

num_user_getad = pd.DataFrame({
    'uid': num_user_getad.index,
    'num_user_getad' : num_user_getad.values
})

num_user_clicked = pd.DataFrame({
    'uid': num_user_clicked.index,
    'num_user_clicked' : num_user_clicked.values
})

In [182]:
X_train = pd.merge(X_train, num_user_clicked, on=['uid'], how='left')
X_test = pd.merge(X_test, num_user_clicked, on=['uid'], how='left')

In [113]:
X_train.fillna(0, inplace=True)

In [114]:
X_test.fillna(0, inplace=True)

In [139]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train[['num_advertise_touser','num_common_interest2', 'num_common_interest1', 'num_common_interest5', 'num_common_topic1'
                        ]].values)
X_train_encoded = scaler.transform(X_train[['num_advertise_touser','num_common_interest2', 'num_common_interest1', 'num_common_interest5', 
                                                'num_common_topic1']].values)

X_test_encoded = scaler.transform(X_test[['num_advertise_touser','num_common_interest2', 'num_common_interest1', 'num_common_interest5', 
                                                 'num_common_topic1']].values)

X_train_encoded= np.hstack((X_train_encoded, ct_trains))
X_test_encoded= np.hstack((X_test_encoded, ct_tests))

# , 'num_common_interest2', 'num_common_interest1', 'num_common_interest5' 'num_advertise_touser' 'ration_num_LBS'
# X_train_encoded = ct_trains
# X_test_encoded = ct_tests

In [140]:
oc_encoder = OneHotEncoder()
for feature in one_hot_feature:
    oc_encoder.fit(data[feature].reshape(-1, 1))
    train_a=oc_encoder.transform(X_train[feature].values.reshape(-1, 1))
    X_train_encoded = sparse.hstack((X_train_encoded, train_a))
    test_a=oc_encoder.transform(X_test[feature].values.reshape(-1, 1))
    X_test_encoded = sparse.hstack((X_test_encoded, test_a))

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  This is separate from the ipykernel package so we can avoid doing imports until


In [141]:
X_train_encoded.toarray().shape

(615917, 1181)

In [142]:
vector_feature=['interest1','interest2','interest5','kw1','kw2','topic1','topic2']
ct_encoder = CountVectorizer(min_df=0.0009)
for feature in vector_feature:
    ct_encoder.fit(data[feature])
    train_a = ct_encoder.transform(X_train[feature])
    X_train_encoded = sparse.hstack((X_train_encoded, train_a))
    test_a = ct_encoder.transform(X_test[feature])
    X_test_encoded = sparse.hstack((X_test_encoded, test_a))

In [143]:
# y_train = np.array(target).squeeze()
clf = lgb.LGBMClassifier(
        boosting_type='gbdt', num_leaves=31, reg_alpha=0.0, reg_lambda=0.9,
        max_depth=-1, n_estimators=500, objective='binary',
        subsample=0.7, colsample_bytree=0.8, subsample_freq=1,
        learning_rate=0.05, min_child_weight=60, random_state=2018, n_jobs=-1
    )
clf.fit(X_train_encoded, y_train, eval_set=[(X_test_encoded, y_test)], eval_metric='auc',early_stopping_rounds=50)

[1]	valid_0's auc: 0.629013
Training until validation scores don't improve for 50 rounds.
[2]	valid_0's auc: 0.635385
[3]	valid_0's auc: 0.639865
[4]	valid_0's auc: 0.640183
[5]	valid_0's auc: 0.640214
[6]	valid_0's auc: 0.641537
[7]	valid_0's auc: 0.642981
[8]	valid_0's auc: 0.642897
[9]	valid_0's auc: 0.644709
[10]	valid_0's auc: 0.648281
[11]	valid_0's auc: 0.650826
[12]	valid_0's auc: 0.652656
[13]	valid_0's auc: 0.65357
[14]	valid_0's auc: 0.655097
[15]	valid_0's auc: 0.655379
[16]	valid_0's auc: 0.656759
[17]	valid_0's auc: 0.657668
[18]	valid_0's auc: 0.658465
[19]	valid_0's auc: 0.658792
[20]	valid_0's auc: 0.658919
[21]	valid_0's auc: 0.659314
[22]	valid_0's auc: 0.660638
[23]	valid_0's auc: 0.661251
[24]	valid_0's auc: 0.661942
[25]	valid_0's auc: 0.662703
[26]	valid_0's auc: 0.663195
[27]	valid_0's auc: 0.663696
[28]	valid_0's auc: 0.663988
[29]	valid_0's auc: 0.664564
[30]	valid_0's auc: 0.66496
[31]	valid_0's auc: 0.665408
[32]	valid_0's auc: 0.665685
[33]	valid_0's auc: 0

[277]	valid_0's auc: 0.711644
[278]	valid_0's auc: 0.711705
[279]	valid_0's auc: 0.711749
[280]	valid_0's auc: 0.711808
[281]	valid_0's auc: 0.711927
[282]	valid_0's auc: 0.711955
[283]	valid_0's auc: 0.712005
[284]	valid_0's auc: 0.712026
[285]	valid_0's auc: 0.712029
[286]	valid_0's auc: 0.712084
[287]	valid_0's auc: 0.712099
[288]	valid_0's auc: 0.71213
[289]	valid_0's auc: 0.712137
[290]	valid_0's auc: 0.71219
[291]	valid_0's auc: 0.712194
[292]	valid_0's auc: 0.712284
[293]	valid_0's auc: 0.712289
[294]	valid_0's auc: 0.712325
[295]	valid_0's auc: 0.712329
[296]	valid_0's auc: 0.712307
[297]	valid_0's auc: 0.712308
[298]	valid_0's auc: 0.712305
[299]	valid_0's auc: 0.712398
[300]	valid_0's auc: 0.712452
[301]	valid_0's auc: 0.712529
[302]	valid_0's auc: 0.712608
[303]	valid_0's auc: 0.71266
[304]	valid_0's auc: 0.712722
[305]	valid_0's auc: 0.712715
[306]	valid_0's auc: 0.7127
[307]	valid_0's auc: 0.712761
[308]	valid_0's auc: 0.712799
[309]	valid_0's auc: 0.712818
[310]	valid_0's

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=0.8,
        learning_rate=0.05, max_depth=-1, min_child_samples=20,
        min_child_weight=60, min_split_gain=0.0, n_estimators=500,
        n_jobs=-1, num_leaves=31, objective='binary', random_state=2018,
        reg_alpha=0.0, reg_lambda=0.9, silent=True, subsample=0.7,
        subsample_for_bin=200000, subsample_freq=1)

In [115]:
def get_common_interest(type_name, ratio):
    num_adid = data_clicked['aid'].value_counts().sort_index().index
    num_aid_clicked = dict(data_clicked['aid'].value_counts().sort_index())
    num_user_clicksameAd_interest = data_clicked.groupby('aid')[type_name].value_counts()
    dict_interest = {}
    for adid in num_adid:
        dict_buf = {}
        for interest in num_user_clicksameAd_interest.items():
            index = interest[0]
            if index[0] == adid:
                number = interest[1]
                detail = index[1]
                detail = detail.split(' ')
                for det in detail:
                    if det not in dict_buf:
                        dict_buf[det] = number
                    else:
                        dict_buf[det] += number
        dict_interest[adid] = dict_buf
    dict_common_interest = []
    for adid, dict_inter in dict_interest.items():
        dict_common_buf = {}
        dict_common_buf['aid'] = adid
        common_inter = []
        ad_total = num_aid_clicked[adid] - dict_inter.get('-1', 0)
        if '-1' in dict_inter:
            dict_inter.pop('-1')
        for id_inter, num in dict_inter.items():
            if num >= ad_total*ratio:
                common_inter.append(id_inter)
        str_name = 'common_'+type_name
        dict_common_buf[str_name] = common_inter
        dict_common_interest.append(dict_common_buf)
    return dict_common_interest

获取相同的兴趣ID2

In [116]:
dict_common_interest2 = get_common_interest('interest2', 0.25)
df_common_interest2 = pd.DataFrame(dict_common_interest2)
X_train = pd.merge(X_train, df_common_interest2, on=['aid'], how='left')
X_test = pd.merge(X_test, df_common_interest2, on=['aid'], how='left')

获取相同的兴趣ID1

In [117]:
dict_common_interest1 = get_common_interest('interest1', 0.25)
df_common_interest1 = pd.DataFrame(dict_common_interest1)
X_train = pd.merge(X_train, df_common_interest1, on=['aid'], how='left')
X_test = pd.merge(X_test, df_common_interest1, on=['aid'], how='left')

获取相同的兴趣ID5

In [118]:
dict_common_interest5 = get_common_interest('interest5', 0.25)
df_common_interest5 = pd.DataFrame(dict_common_interest5)
X_train = pd.merge(X_train, df_common_interest5, on=['aid'], how='left')
X_test = pd.merge(X_test, df_common_interest5, on=['aid'], how='left')

In [119]:
X_train['num_common_interest2'] = [len(set(i.split(' ')).intersection(set(j))) / len(j) for i, j in X_train[['interest2', 'common_interest2']].values]
X_test['num_common_interest2'] = [len(set(i.split(' ')).intersection(set(j))) /len(j) for i, j in X_test[['interest2', 'common_interest2']].values]

In [120]:
X_train['num_common_interest1'] = [len(set(i.split(' ')).intersection(set(j))) / len(j) for i, j in X_train[['interest1', 'common_interest1']].values]
X_test['num_common_interest1'] = [len(set(i.split(' ')).intersection(set(j))) / len(j) for i, j in X_test[['interest1', 'common_interest1']].values]

In [121]:
X_train['num_common_interest5'] = [len(set(i.split(' ')).intersection(set(j))) / len(j) for i, j in X_train[['interest5', 'common_interest5']].values]
X_test['num_common_interest5'] = [len(set(i.split(' ')).intersection(set(j))) / len(j) for i, j in X_test[['interest5', 'common_interest5']].values]

In [15]:
X_train['num_common_all'] = X_train[['num_common_interest5', 'num_common_interest2', 'num_common_interest1']].sum(axis=1)
X_test['num_common_all'] = X_test[['num_common_interest5', 'num_common_interest2', 'num_common_interest1']].sum(axis=1)

提取关键字1,2

In [122]:
dict_common_kw1 = get_common_interest('kw1', 0.05)
df_common_kw1 = pd.DataFrame(dict_common_kw1)

In [123]:
X_train = pd.merge(X_train, df_common_kw1, on=['aid'], how='left')
X_test = pd.merge(X_test, df_common_kw1, on=['aid'], how='left')

In [124]:
X_train['num_common_kw1'] = [len(set(i.split(' ')).intersection(set(j))) / (len(j)+1) for i, j in X_train[['kw1', 'common_kw1']].values ]
X_test['num_common_kw1'] = [len(set(i.split(' ')).intersection(set(j))) /  (len(j)+1) for i, j in X_test[['kw1', 'common_kw1']].values ]

In [125]:
dict_common_kw2 = get_common_interest('kw2', 0.05)
df_common_kw2 = pd.DataFrame(dict_common_kw2)
X_train = pd.merge(X_train, df_common_kw2, on=['aid'], how='left')
X_test = pd.merge(X_test, df_common_kw2, on=['aid'], how='left')
X_train['num_common_kw2'] = [len(set(i.split(' ')).intersection(set(j))) / (len(j)+1) for i, j in X_train[['kw2', 'common_kw2']].values ]
X_test['num_common_kw2'] = [len(set(i.split(' ')).intersection(set(j))) /  (len(j)+1) for i, j in X_test[['kw2', 'common_kw2']].values ]

提取主题1,2

In [126]:
dict_common_topic1 = get_common_interest('topic1', 0.1)
df_common_topic1 = pd.DataFrame(dict_common_topic1)
X_train = pd.merge(X_train, df_common_topic1, on=['aid'], how='left')
X_test = pd.merge(X_test, df_common_topic1, on=['aid'], how='left')
X_train['num_common_topic1'] = [len(set(i.split(' ')).intersection(set(j))) / (len(j)+1) for i, j in X_train[['topic1', 'common_topic1']].values ]
X_test['num_common_topic1'] = [len(set(i.split(' ')).intersection(set(j))) /  (len(j)+1) for i, j in X_test[['topic1', 'common_topic1']].values ]

In [127]:
dict_common_topic2 = get_common_interest('topic2', 0.05)
df_common_topic2 = pd.DataFrame(dict_common_topic2)
X_train = pd.merge(X_train, df_common_topic2, on=['aid'], how='left')
X_test = pd.merge(X_test, df_common_topic2, on=['aid'], how='left')
X_train['num_common_topic2'] = [len(set(i.split(' ')).intersection(set(j))) / (len(j)+1) for i, j in X_train[['topic2', 'common_topic2']].values ]
X_test['num_common_topic2'] = [len(set(i.split(' ')).intersection(set(j))) /  (len(j)+1) for i, j in X_test[['topic2', 'common_topic2']].values ]

In [132]:
X_train[X_train['num_common_kw2'] != 0]

,aid,uid,advertiserId,campaignId,creativeId,creativeSize,adCategoryId,productId,productType,LBS,...,num_common_interest1,num_common_interest5,common_kw1,num_common_kw1,common_kw2,num_common_kw2,common_topic1,num_common_topic1,common_topic2,num_common_topic2
1,543,74408984,15,108,130,5,7,0,1,147,...,0.666667,0.897436,[562294],0.000000,"[66915, 11395, 79112, 8546, 105115, 18185, 851...",0.100000,"[523, 9826]",0.333333,"[2979, 5641, 5115]",0.000000
4,1021,82403497,7,101,117,1,10,3,2,5,...,0.750000,0.535714,"[2268, 8561, 279516]",0.000000,"[1373, 102474, 846, 87384, 81211, 45649, 11571...",0.086957,[9826],0.000000,"[8045, 7723, 1553, 1911, 9018, 7621, 1933, 700...",0.000000
5,389,21293459,36,130,126,8,6,0,0,77,...,0.000000,0.000000,"[8561, 53354, 338851, 605930]",0.000000,"[11395, 79112, 66915, 18185]",0.200000,[9826],0.000000,[6945],0.000000
8,1672,15260609,5,68,83,2,18,3,2,367,...,0.222222,0.000000,"[709037, 555710, 53354, 411962, 338851, 369678...",0.100000,"[11547, 5685, 66915, 108775, 30380, 19007, 565...",0.029412,"[9826, 644]",0.000000,"[1276, 1471, 3114, 9145, 2438, 1553, 7621, 895...",0.000000
9,191,62714953,61,44,7,2,16,25,3,61,...,0.444444,0.181818,"[411962, 338851, 53354, 562294]",0.000000,"[80452, 55867, 90034, 117548, 18185, 102474, 1...",0.200000,"[7684, 9826, 644]",0.250000,[],0.000000
10,1716,50609177,22,80,94,2,10,3,2,332,...,0.444444,0.000000,"[8561, 279516, 53354, 64588]",0.000000,"[56438, 66915, 79676, 85039, 18185, 93916, 299...",0.090909,[9826],0.000000,"[9171, 1911, 263, 7723, 895]",0.000000
15,70,57166624,5,8,0,1,10,3,2,37,...,0.444444,0.423077,"[369678, 53354, 8561, 605677, 517534]",0.166667,"[11547, 5685, 85039, 43977, 15941, 109642, 977...",0.058824,"[9826, 644]",0.000000,"[1471, 1553, 263, 7723, 1911, 6552, 895, 8125,...",0.000000
19,173,62445910,25,93,27,2,23,10,2,361,...,0.300000,0.166667,[53354],0.000000,"[102474, 52864, 85039, 11395, 79112, 37979, 66...",0.250000,"[9826, 7684]",0.000000,"[982, 6945, 9968, 8005]",0.000000
20,302,3834034,51,136,155,9,6,0,0,518,...,0.882353,0.518519,"[338851, 8561, 580549, 605930]",0.200000,"[85039, 102474, 11395, 79112, 80263, 28206, 52...",0.266667,"[6661, 9826, 6939]",0.500000,[],0.000000
22,70,73037277,5,8,0,1,10,3,2,434,...,0.444444,0.346154,"[369678, 53354, 8561, 605677, 517534]",0.000000,"[11547, 5685, 85039, 43977, 15941, 109642, 977...",0.117647,"[9826, 644]",0.333333,"[1471, 1553, 263, 7723, 1911, 6552, 895, 8125,...",0.100000


In [66]:
# X_train.drop(['common_topic1_x','common_topic1_y'] ,axis=1, inplace=True)
# X_test.drop(['common_topic1_x','common_topic1_y'] ,axis=1, inplace=True)

# X_test.drop(['common_interest2_x','common_interest1_x','interest2_new','interest1_new','common_interest2_y'
#              ,'common_interest1_y','common_interest2','common_interest1'], axis=1, inplace=True)

# X_test.drop(['num_common_interest','common_interest1_x','num_common_interest1','common_interest1_y','common_interest1','common_interest2_y'],
#             axis=1, inplace=True)
# X_train.drop(['num_common_interest','common_interest1_x','num_common_interest1','common_interest1_y','common_interest1','common_interest2_y'],
#             axis=1, inplace=True)

In [206]:
for i, j in X_train[['interest2', 'common_interest2']].values:
    print(i, j)
    break

22 1 70 ['30', '73', '70']


In [83]:
train_interest1_new = []
for i in train_user_interest1.values:
    inter2_buf = []
    int2 = i[0].split(' ')
    common = i[1]
    for det in int2:
        if det in common:
            inter2_buf.append(det)
    inter2_buf.extend(int2)
    train_interest1_new.append(inter2_buf)

In [84]:
test_interest1_new = []
for i in test_user_interest1.values:
    inter2_buf = []
    int2 = i[0].split(' ')
    common = i[1]
    for det in int2:
        if det in common:
            inter2_buf.append(det)
    inter2_buf.extend(int2)
    test_interest1_new.append(inter2_buf)

In [85]:
train_interest2_new = [' '.join(m) for m in train_interest2_new]
X_train['interest2_new'] = train_interest2_new

In [86]:
test_interest2_new = [' '.join(m) for m in test_interest2_new]
X_test['interest2_new'] = test_interest2_new

In [87]:
train_interest1_new = [' '.join(m) for m in train_interest1_new]
X_train['interest1_new'] = train_interest2_new

In [88]:
test_interest1_new = [' '.join(m) for m in test_interest1_new]
X_test['interest1_new'] = test_interest1_new

In [89]:
concat_new = pd.concat([X_train[['uid', 'interest2_new']], X_test[['uid', 'interest2_new']]])

In [91]:
concat_new1 = pd.concat([X_train[['uid', 'interest1_new']], X_test[['uid', 'interest1_new']]])